In [1]:
import pandas as pd

In [2]:
with open("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt") as f:
    for line in f:
        print(line)
        break

{"task_id": "4a06139e", "domain": "UPDATE_CALENDAR", "bot_prompt": "Schedule the user's meeting request", "bot_role": "You are a bot designed to help schedule meetings on a calendar. ", "user_prompt": " You have a meeting saved for March 24th. Ask the chatbot to delete the meeting", "user_role": "You are interacting with a meeting scheduling bot"}



In [3]:
tasks = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/tasks.txt", lines=True)

In [4]:
dialog = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/ORDER_PIZZA.txt", lines=True)

In [23]:
import glob

txtfiles = []
for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/metalwoz-v1/dialogues/*.txt"):
    txtfiles.append(file)

In [24]:
metalwoz = pd.read_json(txtfiles[0], lines=True)
for txt in txtfiles[1:]:
    tmp = pd.read_json(txt, lines=True)
    metalwoz = pd.concat([metalwoz,tmp])


In [46]:
import glob

txtfiles = []
for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/*.json"):
    df = pd.read_json(file)
    break

In [58]:
a = "This is the first context"

In [154]:
schema = pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/train/schema.json")
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/dev/schema.json")])
schema = pd.concat([schema, pd.read_json("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/test/schema.json")])

In [126]:
slots = []
for idx, row in df.iterrows():
#     print(row)
    service = row['service_name']
    for i in row.slots:
        slots.append(service+"_"+i['name'])
#     for i in row.intents:
#         slots.extend([service+"_"+j for j in i['required_slots']])
#         slots.extend([service+"_"+j for j in list(i['optional_slots'].keys())])
    

In [534]:
import glob

txtfiles = []
dialog = None
for i in ['train', 'dev', 'test']:
    for file in glob.glob("/Users/jarana/workspace/google-research/schema_guided_dst/data/dstc8-schema-guided-dialogue/%s/*.json" % i):
        txtfiles.append(file)
        dialog = pd.concat([dialog, pd.read_json(file)])

In [535]:
domains = []
for i in dialog.services.tolist():
    try:
        if len(i) == 1:
            domains.append(i[0])
    except:
        continue

In [539]:
from sklearn.model_selection import train_test_split
trainIds, testIds = train_test_split(list(set(domains)), test_size=0.3, random_state=2020)

In [520]:
prop2desc = {}
for idx, s in schema.iterrows():
    service = s.service_name
    for i in s.slots:
        prop2desc[service+"_"+i['name']] = i['description']


In [521]:
len(prop2desc), schema.service_name.nunique()

(365, 45)

In [541]:
train_data = []
test_data = []

qid = 1
for idx, row in dialog.iterrows():
    frames = row['turns']
    context = []
    
    if str(frames)=='nan':
        continue
        
    for f in frames:
        service = f['frames'][0]['service']
        if len(f['frames']) == 1:
            utterance = f['utterance']
            if len(f['frames'][0]['slots']) != 0:
                terms = []
                for s in f['frames'][0]['slots']:
                    _context = " ".join(context)
                    answer_start = len(_context) + s["start"] + 1
                    new_answer = utterance[:s["start"]] + s['slot'].replace("_", " ") + utterance[s["exclusive_end"]+1:]
                    _context += " " + new_answer
                    _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
                                                          'question': prop2desc[service+"_"+s['slot']], 'answers':[{'text':s['slot'].replace("_"," "), 
                                                           'answer_start': answer_start}]}]}
                    if service in trainIds:
                        train_data.append(_json)
                    else:
                        test_data.append(_json)
                    qid += 1
            context.append(utterance)


In [542]:
len(train_data), len(test_data)

(191224, 122204)

In [545]:
### Save as a JSON file
import json
with open('data/dialog/train.json', 'w') as f:
    json.dump(train_data, f)
import json
with open('data/dialog/test.json', 'w') as f:
    json.dump(test_data, f)

In [475]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging
import torch

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('bert', 'bert-large-uncased-whole-word-masking-finetuned-squad', use_cuda=torch.cuda.is_available()
, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

# Train the model with JSON file
model.train_model('data/dialog/train.json')

# The list can also be used directly
# model.train_model(train_data)

# Evaluate the model. (Being lazy and evaluating on the train data itself)
result, text = model.eval_model('data/dialog/train.json')

print(result)
print(text)

print('-------------------')

# Making predictions using the model.
to_predict = [{'context': 'This is the context used for demonstrating predictions.', 'qas': [{'question': 'What is this context?', 'id': '0'}]}]

print(model.predict(to_predict))

IndexError: list index out of range